In [6]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

# Load the dataset
file_path = "F:/FYP project/FYP(Flood)/Real Data/Flood_fyp_data.csv"  # Replace with your file path
data = pd.read_csv(file_path)

# Handle NaN values using forward-fill
data.fillna(method='ffill', inplace=True)

# Drop 'Date' and 'Hour' columns if they exist
data = data.drop(columns=['Date', 'Hour'], errors='ignore')

# Define input features (X) and target values (y)
X = data[['Discharge Rate (cumecs)', 'Rainfall Data (mm)', 'Water Level (m)']].values
y = data[['Next 1 Hour Water Level (m)', 'Next 2 Hours Water Level (m)', 'Next 3 Hours Water Level (m)']].values

# Normalize the input features
scaler_X = MinMaxScaler(feature_range=(0, 1))
X_scaled = scaler_X.fit_transform(X)

# Reshape the data to be 3D for LSTM input
X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))  # 3D shape for LSTM

# Split dataset into training (70%), validation (15%), and testing (15%)
X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Initialize the LSTM model
model = Sequential()

# LSTM layers
model.add(LSTM(units=64, return_sequences=False, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=3))  # 3 outputs (1st, 2nd, 3rd hour predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), verbose=1)

# Predict on the test set
y_test_pred = model.predict(X_test)

Epoch 1/50


C:\Users\Janith\AppData\Local\Temp\ipykernel_40168\3711216904.py:17: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)
c:\Users\Janith\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


113/113 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 8.7007 - val_loss: 1.9748
Epoch 2/50
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.3912 - val_loss: 0.3536
Epoch 3/50
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2962 - val_loss: 0.2407
Epoch 4/50
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2054 - val_loss: 0.1652
Epoch 5/50
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1320 - val_loss: 0.1205
Epoch 6/50
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1117 - val_loss: 0.0924
Epoch 7/50
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0847 - val_loss: 0.0783
Epoch 8/50
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0776 - val_loss: 0.0720
Epoch 9/50
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0779 - val_loss: 0.0699
Epoch 10/50
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0678 - val_loss: 0.0691
Epoch 11/50
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0720 - val_loss: 0.0693
Epoch 12/50
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.

In [7]:
# Example new data (Discharge Rate, Rainfall, and Water Level)
new_data = np.array([[262., 0.1, 6.81]])  # Example input data

# Scale the new data using the same scaler used for training
new_data_scaled = scaler_X.transform(new_data)

# Reshape the new data for LSTM (samples, timesteps, features)
new_data_scaled = new_data_scaled.reshape((new_data_scaled.shape[0], 1, new_data_scaled.shape[1]))

# Make predictions using the trained model
new_predictions = model.predict(new_data_scaled)

# Print the predictions for the next 1, 2, and 3 hours
print("\nPredictions for new data:")
for i, target in enumerate(['Next 1 Hour', 'Next 2 Hours', 'Next 3 Hours']):
    print(f"{target} Water Level: {new_predictions[0, i]:.2f} m")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step

Predictions for new data:
Next 1 Hour Water Level: 6.71 m
Next 2 Hours Water Level: 6.67 m
Next 3 Hours Water Level: 6.65 m


In [12]:
import numpy as np

# Predefined flood levels
flood_levels = {
    'Alert Flood Level': 5.2,
    'Minor Flood Level': 7.5,
    'Major Flood Level': 9.5,
    'Critical Flood Level': 10.5
}

# Function to predict and generate alert messages
def generate_flood_alert(new_data, model, scaler_X, scaler_y):
    """
    Predict water levels for the next 3 hours and generate an alert message.
    
    Args:
    - new_data (np.array): Input data for prediction (e.g., Discharge Rate, Rainfall, Water Level)
    - model: Trained LSTM model
    - scaler_X: Scaler used for input features
    - scaler_y: Scaler used for target (water levels)
    
    Returns:
    - dict: Predicted water levels and corresponding alert message
    """
    # Scale the input data using the scaler for X
    new_data_scaled = scaler_X.transform(new_data)
    
    # Reshape for LSTM input (samples, timesteps, features)
    new_data_scaled = new_data_scaled.reshape((new_data_scaled.shape[0], 1, new_data_scaled.shape[1]))
    
    # Make prediction using the trained LSTM model
    prediction_scaled = model.predict(new_data_scaled)
    
    # Inverse transform the predicted values to get the actual water levels
    prediction = scaler_y.inverse_transform(prediction_scaled)
    
    # Generate alert messages for each hour's prediction
    alerts = []
    for i, target in enumerate(['Next 1 Hour', 'Next 2 Hours', 'Next 3 Hours']):
        predicted_level = prediction[0, i]
        alert_message = f"{target} Prediction: {predicted_level:.2f} m - "
        
        if predicted_level >= flood_levels['Critical Flood Level']:
            alert_message += "Critical Flood Alert! Evacuate immediately!"
        elif predicted_level >= flood_levels['Major Flood Level']:
            alert_message += "Major Flood Alert! Prepare for evacuation."
        elif predicted_level >= flood_levels['Minor Flood Level']:
            alert_message += "Minor Flood Alert! Take precautionary measures."
        else:
            alert_message += "Water level is below alert thresholds."
        
        alerts.append(alert_message)
    
    return alerts

# Example new data (Discharge Rate, Rainfall, and Water Level)
new_data = np.array([[262., 0.1, 6.81]])  # Example input data

# Call the function to predict and generate alerts
alerts = generate_flood_alert(new_data, model, scaler_X, scaler_y)

# Print the generated alerts
for alert in alerts:
    print(alert)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Next 1 Hour Prediction: 0.16 m - Water level is below alert thresholds.
Next 2 Hours Prediction: 0.29 m - Water level is below alert thresholds.
Next 3 Hours Prediction: -0.19 m - Water level is below alert thresholds.
